## A Guide to Digitizing Historical Maps Using QGIS, Python, and Lightroom

Authors: Haicheng Xu & Idaliya Grigoryeva

### Intro

This guide will walk you through the process of digitizing a historical map, using a map of Indonesia as our example. Our goal is to extract the road network to create a friction surface, representing the travel cost between any two points. We specifically chose a map where the roads are distinctly red, allowing for easier extraction.

3 steps for digitizing: 
1. In Lightroom, desaturate all colors except red to isolate the roads.
2. Use the map_to_bw function to convert the processed map to black and white.
3. Import the black-and-white map into QGIS, convert the raster to points, and connect the points using the "nearest points to path" tool.


Map Download:

You can access the Indonesia map here: [Indonesia Map](https://digitalcollections.universiteitleiden.nl/view/item/814037?solr_nav%5Bid%5D=425f4cdcd85c99e9fcfd&solr_nav%5Bpage%5D=0&solr_nav%5Boffset%5D=11) (this map is from Leiden University Library digital collection)

Note: Please download the "Original Master" version of the map for the best result.

### 1. Extracting red roads with Lightroom

<small>* Notes about Lightroom:

- Adobe Lightroom is a popular photo editing and management software used by photographers and creatives to enhance and organize their images. It offers powerful tools for adjusting exposure, color, sharpness, and more, as well as features for cataloging and organizing large photo libraries.

- Adobe Lightroom offers both a free and a paid version. The free version is a mobile app, but the desktop version I will be using for this tutorial is the paid version.

- This guide assumes you have some prior experience with Lightroom. If you're new to the software, I suggest starting with beginner tutorials available on YouTube or Adobe’s website.

Link to download Lightroom [here](https://www.adobe.com/products/photoshop-lightroom/campaign/pricing.html?gclid=CjwKCAjwuMC2BhA7EiwAmJKRrPLp1CHwbcyfkBg5m4mQuitbiKv8sRZDBmG98TKa3WOgaapQQS2F2xoC2QEQAvD_BwE&sdid=BDDS3CR2&mv=search&mv2=paidsearch&ef_id=CjwKCAjwuMC2BhA7EiwAmJKRrPLp1CHwbcyfkBg5m4mQuitbiKv8sRZDBmG98TKa3WOgaapQQS2F2xoC2QEQAvD_BwE:G:s&s_kwcid=AL!3085!3!677050899114!e!!g!!adobe%20lightroom!1712238382!67643557900&mv=search&gad_source=1)

</small>

##### What we're trying to achieve in step 1 and why: 

The map image you’ve downloaded is made up of pixels, with each pixel's color represented by three values: red (r), green (g), and blue (b). This system, known as RGB, is the foundation of digital color representation. Each value ranges from 0 to 255, and by combining different intensities of red, green, and blue, any color can be created.

For our digitization process, we’re interested only in the roads, which are distinctly marked in red on the map. To simplify the extraction of these roads, we need to eliminate the influence of all other colors, leaving only the red pixels. This is why we desaturate all colors except for red in Lightroom—by doing so, we isolate the roads, making them easier to convert into a black-and-white image in the next step.

Here are the steps to extracting red pixels from the map:

0. Import the Map into Lightroom: Start by importing the map image into Lightroom.

1. Eliminate the saturation for all colors except for red.

2. Boost overall saturation to 100 to make it easier for python program (next step) to identify the red.

    <img src="Images/lr-1.jpg" alt="Saturation adjustments" width="500"/>



3. Decrease the saturation of midtone and shadow areas to remove some red noise from the yellow background of the map.

4. Increase the saturation of highlight areas to make the red more prominent.

    <img src="Images/lr-2.jpg" alt="Color grading adjustments" width="500"/>


5. Using color noise reduction to further reduce the red noise due to the yellow background.

    <img src="Images/lr-3.jpg" alt="Image Description" width="500"/>


6. Export the image from lightroom and select large size. 

    <img src = 'Images/lr-5.jpg' alt = 'Lr export' width="500">

### 2. Convert processed map to black and white with Python

Having obtained the processed image from Step 1, where we've isolated the red pixels (representing roads), we will now convert the image to black and white using Python. The purpose of this step is to further simplify the image by turning the red pixels into white and all other pixels into black. This binary (black and white) image makes it easier to extract the road network as a shapefile in QGIS.

What the function map_to_bw does:

- Input: The function takes the file path of the processed image as input (image_path).
- Output: It outputs a black-and-white image, saved as 'Sulawesi_bw.png' in the specified directory.
- Middle: The function processes each pixel in the image, checking if it’s red. If it is, the pixel is turned white; otherwise, it’s turned black.


<small> * Notes about Python:

- Python is a versatile programming language widely used for data processing, automation, and scripting tasks in various fields, including geospatial analysis.

- In this step, we use Python to process the image by isolating specific color pixels and converting the map to black and white, making it easier to extract road networks in QGIS.

- This guide assumes basic familiarity with Python. If you’re new to Python, consider exploring introductory tutorials online.

Link to download Python [here](https://www.python.org/downloads/).

</small>

In [5]:
from PIL import Image

In [10]:
from PIL import Image

def map_to_bw(image_path):
    # Open the image from the specified path
    im = Image.open(image_path)

    # Ensure the image is in RGB mode, which represents each pixel's color with three values: red, green, and blue
    im = im.convert("RGB")

    # Get a pixel access object, allowing us to read and modify the image's pixels
    pix = im.load()

    # Iterate over every pixel in the image using nested loops
    for x in range(im.width):  # Loop through each column (width)
        for y in range(im.height):  # Loop through each row (height)
            # Retrieve the RGB values of the current pixel
            r, g, b = pix[x, y]
            
            # Check if the pixel is predominantly red by comparing the red value to green and blue
            # The condition checks if the red value is significantly higher than the green and blue
            if (r - g + r - b) > 80:
                # If the pixel is red, change it to white (RGB: 256, 256, 256)
                pix[x, y] = (256, 256, 256)
            else:
                # If the pixel is not red (i.e., it's grey or another color), change it to black (RGB: 0, 0, 0)
                pix[x, y] = (0, 0, 0)

    # Save the modified image as 'Sulawesi_bw.png' in the specified directory
    im.save('Images/Sulawesi_bw.png')


Uncomment the code below to run the map_to_bw function:

In [11]:
# map_to_bw('Images/Sulawesi_processed.jpg')

### 3. Digitizing b&w map using QGIS 

<small> *Notes about QGIS:

- QGIS is a powerful open-source Geographic Information System (GIS) software used by professionals and enthusiasts alike for mapping and spatial analysis. It offers a wide range of tools for visualizing, analyzing, and editing geographic data, making it a go-to choice for tasks like creating maps, conducting spatial analysis, and managing geospatial data.

- QGIS is freely available, and its open-source nature means it’s constantly being improved by a global community of developers. This tutorial assumes you have some basic experience with QGIS. If you’re new to the software, I recommend reading the [QGIS documentation](https://docs.qgis.org/3.34/en/docs/user_manual/)

You can download QGIS [here](https://www.qgis.org/download/)


In this step, we use QGIS to georeference the processed map from step 2, overlay it with OpenStreetMap (OSM) layers, and convert the road network from raster to vector data. This process involves aligning the historical map with real-world coordinates and extracting the road information for further analysis.

Here are the steps to create vector road data using QGIS:

1. Download QuickMapService plugin if you don’t have it 
2. Open a OSM standard layer

    <img src = "Images/osm_layer.jpg"  alt="opening osm layer" width="500"/>


3. Georeference the Sulawesi_bw.png onto OSM standard using this method: https://www.youtube.com/watch?v=jKLBFddpTGI

    Note: add 10-20 reference points throughout the North, South, East, and West corners of the map for the best result

4. Click into layer property by double clicking the Sulawesi_bw layer and change its transparency to 50%.

5. Resulting Georeferenced map of Sulawesi:

    <img src = "Images/georeferenced_result-1.jpg"  alt="georeferencing-1" width="500"/>

    <img src = "Images/georeferenced_result-2.jpg"  alt="georeferencing-2" width="500"/>

    <img src = "Images/georeferenced_result-3.jpg"  alt="georeferencing-3" width="500"/>



Now that we've georeferenced the map onto OSM, we will convert the jpeg map (raster) to vector points in steps 6-9:

6. Using raster pixels to points function to convert roads in white to vector/point data

    Note: [Link](https://www.youtube.com/watch?v=3UNz09UyXa8) to a video for steps 6-9

7. Run the function with the georeferenced map from step 5 as input

    <img src = "Images/rptp-1.jpg"  alt="raster pixels to points 1" width="500"/>

    <img src = "Images/rptp-2.jpg"  alt="raster pixels to points 2" width="500"/>


After running raster pixels to points, you'll probably be overwhelmed by the countless points covering the entire map. This is because the function converts every pixel to a point regardless of its color (black or white). To remove all the black point and leaving only the white roads:

8. Right click on vector points → properties →  Symbology → categorized (from the top drop-down menu → select value (from the value drop-down menu) → classify

9. After classifying, go to sources (we’re still in properties menu) → provide feature filter → query builder → type "VALUE" = 255 → OK

    Result:

    <img src = "Images/rptp-3.jpg"  alt="raster pixels to points result" width="500"/>

    South West Corner:

    <img src = "Images/rptp-4.jpg"  alt="raster pixels to points result SW" width="500"/>



If you zoom into the map, you'll realize some roads are wider than others, and most raods has a width of more than 1 pixel. However, we only want the road to be 1 pixel wide. To achieve that, we can use geodesic points to decimate function to reduce the duplicated points

10. In the top toolbar: Vector → Shape Tools → Geodesic Geometry Simplification → Geodesic Point Decimate

    <img src = "Images/gpd-1.jpg"  alt="raster pixels to points result SW" width="500"/>


    <img src = "Images/gpd-3.jpg"  alt="raster pixels to points result SW" width="500"/>

* Note: The best parameter for minimum distance between point differs between maps. 5000 meters works best for this map.


11. Result:

    <img src = "Images/gpd-4.jpg"  alt="raster pixels to points result SW" width="500"/>

    <img src = "Images/gpd-5.jpg"  alt="raster pixels to points result SW" width="500"/>

    <img src = "Images/gpd-6.jpg"  alt="raster pixels to points result SW" width="500"/>




Finally, we will connect the points with lines using the nearest points to path function

12. Find nearest points to path in processing toolbox → use the point layer (5km) from step 11 → parameters: Max Distance = 3000, Max points = 0    

    <img src = "Images/ptp-1.jpg"  alt="nearest points to path 1" width="500"/>

<small> * Note * 

1. This took my m2 macbook pro 13 about 2-3 hours to run
2. The optimal parameters for max distance and max point for different maps will likely be different. I've tried several different configurations and found max distance = 3000 and max points = 0 works best for my map. I recommend playing with these two parameters with 3000,0 as baseline. Also, I suggest running it overnight for the best use of your time :)

</small> 

13. Result:

    <img src = "Images/ptp-2.jpg"  alt="nearest points to path 1" width="500"/>

    <img src = "Images/ptp-3.jpg"  alt="nearest points to path 1" width="500"/>

    <img src = "Images/ptp-4.jpg"  alt="nearest points to path 1" width="500"/>



Manual Cleaning

On the map of Sulawesi, you might notice some pixels in the middle of the ocean. Some of them are from the legend, while others are from the edges of the map. The final step is to select and remove these pixels manually.

14. Select the pixels you want to remove using the select feature by area tool on the top tool bar. Select the first group by drawing a rectangle around them. To select additional groups of pixels, hold down the shift key while making your selection.

    <img src = "Images/cleaning-1.jpg"  alt="nearest points to path 1" width="500"/>

    <img src = "Images/cleaning-2.jpg"  alt="nearest points to path 1" width="500"/>
    



15. After you have selected the pixels they will be highlighted yellow. Click on Toggle Edit, then click Delete Selected. Finally, click Toggle Edit again and click save to save your deletion.

    <img src = "Images/cleaning-4.jpg"  alt="nearest points to path 1" width="500"/>

    <img src = "Images/cleaning-5.jpg"  alt="nearest points to path 1" width="500"/>
    
    <img src = "Images/cleaning-6.jpg"  alt="nearest points to path 1" width="500"/>

Woohoo! We're done :)

### Road Digitization Results 

I overlayed the road on top of the raster map to examine the accuracy of the digitization process:

<img src = "Images/result-1.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-se.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-sw.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-ne.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-nw.jpg"  alt="nearest points to path 1" width="500"/>

Accurate parts:

<img src = "Images/result-good1.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-good2.jpg"  alt="nearest points to path 1" width="500"/>

<img src = "Images/result-good3.jpg"  alt="nearest points to path 1" width="500"/>


Inaccurate parts:

1. Roads that are represented by dotted lines are not connected well. 

    <img src = "Images/result-p1.jpg"  alt="nearest points to path 1" width="500"/>

2. Intersections of multiple roads are not represented correctly.

    <img src = "Images/result-p2.jpg"  alt="nearest points to path 1" width="500"/>

3. Small parts of the main road is disconnected

    <img src = "Images/result-p3.jpg"  alt="nearest points to path 1" width="500"/>
